<a href="https://colab.research.google.com/github/dtht2d/astronomy/blob/main/bispectrum/colabnote/test_SO4_bispectrum_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Function Compute Bispectrum Component**: 

In [2]:
%autosave 30

Autosaving every 30 seconds


In [3]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.6 MB/s eta 0:00:00


In [4]:
pip install sympy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.0 MB/s eta 0:00:00
time: 502 µs (started: 2023-04-13 21:27:36 +00:00)


# Load data

**Input:** cif file
- Set of data, neighbor list, it could be $x, y, z$ coordinate and convert to $\theta_0, \theta, \phi$
- Cell length (unit angstrom)
Note: Chosen $R_{cut}$ needs to divide by the true cell length (i.e: with example dataset we need to divide by 18.7337) since (𝑥,𝑦,𝑧) are fraction with cell dimension (1,1,1), $R_i, R_k$ (center, neighbor): atom radius, measures from center of nucleus to the outermost isolated electron also so need to divide by
-Atom type

**Output:** .json file neighbor atoms list and important values for B calculation 
- Save neigbor atoms list for a choosen center atom as .json file (JSON is a simple and lightweight format for representing data)                         
- ['atom_type', 'X', 'Y', 'Z', 'X_k', 'Y_k', 'Z_k', 'r_ik', 'theta_0', 'theta', 'phi', 'w_ik', 'delta'] 
- Neighbor list excludes center atom itself


In [28]:
import numpy as np
import json
import pandas as pd
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
def get_INPUT_value(center_atom_id:int, r_mu, R_cut, input_file_path:str, output_directory:str, file_type:str):
    """
    Parameters

    """
    if file_type == "cif":
        # DATA PREPARATION
        dico = MMCIF2Dict(input_file_path)
        df_cif = pd.DataFrame.from_dict(dico, orient='index')
        x = df_cif.iloc[-3]
        y = df_cif.iloc[-2]
        z = df_cif.iloc[-1]
        atom_type = df_cif.iloc[-4]
        x_array = np.array(x[0], dtype=float)
        y_array = np.array(y[0], dtype=float)
        z_array = np.array(z[0], dtype=float)
        atom_type_array = np.array(atom_type[0], dtype=str)
        df = pd.DataFrame({"atom_type": atom_type_array, "X": x_array, "Y": y_array, "Z": z_array})
        # Estimate list of potentially atoms in the center cell
        id = center_atom_id
        # id
        x_i = df['X'].iloc[id]
        y_i = df['Y'].iloc[id]
        z_i = df['Z'].iloc[id]
        # print(x_i,y_i,z_i)
        X_array = df['X'].to_numpy()
        Y_array = df['Y'].to_numpy()
        Z_array = df['Z'].to_numpy()
        X_k_array = X_array - x_i
        Y_k_array = Y_array - y_i
        Z_k_array = Z_array - z_i
        r_ik = np.sqrt(np.square(X_k_array) + np.square(Y_k_array) + np.square(Z_k_array))
        df['X_k'], df['Y_k'], df['Z_k'], df['r_ik'] = X_k_array, Y_k_array, Z_k_array, r_ik
        # INPUT values
        cell_length = df.iloc[4]  # index row start from 0 _cell_length_a at row 5 index [4]
        #r_mu = 0.0779  # scale atomic radius w.r.t cell length
        #R_cut = 0.25  # scaled value w.r.t cell length (for Si-Si case) 
        #exclude center atom
        mask = (df['r_ik'] + r_mu) <= R_cut
        mask &= df['r_ik'] != 0
        df_ik = df[mask].copy(deep=True)
        # ANGEL CONVERSION
        # theta_0
        r_ik_array = df_ik['r_ik'].to_numpy()  # r_ik from selected neighbors
        r_0_array = np.full((r_ik_array.shape), R_cut)
        theta_0_array = np.pi * (np.divide(r_ik_array, r_0_array))
        # theta
        Z_k_abs_array = np.abs(df_ik['Z_k'].to_numpy())
        theta_array = np.arccos(np.divide(Z_k_abs_array, r_ik_array))
        # phi
        X_k_array = df_ik['X_k'].to_numpy()
        Y_k_array = df_ik['Y_k'].to_numpy()
        phi_array = np.arctan(np.divide(Y_k_array, X_k_array))
        # convert angle to positive value between [0,2pi]
        phi_array_convert = np.mod(phi_array, 2 * np.pi)
        for angle in phi_array_convert:
            if (angle >= 2 * np.pi) and (angle < 0):
                raise ValueError('phi angle in between 0 and 2pi')
        # replace NaN with 0: (code will have error for invalid value center atom values 0/0)_
        df_ik['theta_0'] = theta_0_array
        df_ik['theta_0'] = df_ik['theta_0'].replace(np.nan, 0)
        df_ik['theta'] = theta_array
        df_ik['theta'] = df_ik['theta'].replace(np.nan, 0)
        df_ik['phi'] = phi_array_convert
        df_ik['phi'] = df_ik['phi'].replace(np.nan, 0)
        # array for weight coefficient w.r.t to atom type
        w_ik_arr = np.full((r_ik_array.shape), 1)
        # delta function delta=1 if i and k has the same element type, if not delta =0
        delta = np.full((r_ik_array.shape), 0)
        delta_arr = np.where(df_ik['atom_type'] == df_ik['atom_type'].iloc[0], 1, delta)
        df_ik['w_ik'] = w_ik_arr
        df_ik['delta'] = delta_arr
    # save the DataFrame as a JSON file
        neighbor_list_path = output_directory + '-atom-' + str(center_atom_id) + '-neighbor-list.json'
        df_ik.to_json(neighbor_list_path, orient='records')
        """
        This function is to read in the neighbor list file
        """
        with open(neighbor_list_path, 'r') as f:
            json_data = json.load(f)

            # create a list of dictionaries from the JSON data
            data_list = [dict(row) for row in json_data]

            # extract data from the list of dictionaries
            r_ik_array = np.array([float(row['r_ik']) for row in data_list])
            theta_0_array = np.array([float(row['theta_0']) for row in data_list])
            theta_array = np.array([float(row['theta']) for row in data_list])
            phi_array = np.array([float(row['phi']) for row in data_list])
            w_ik_array = np.array([float(row['w_ik']) for row in data_list])
            delta_array = np.array([float(row['delta']) for row in data_list])
            r_cut_array = np.full((r_ik_array.shape), R_cut)
            # return a dictionary with the extracted data
    else:
        raise ValueError(f"Unsupported file type: {file_type}")
    return {
            'r_ik': r_ik_array,
            'theta_0': theta_0_array,
            'theta': theta_array,
            'phi': phi_array,
            'w_ik': w_ik_array,
            'delta': delta_array,
            'r_cut': r_cut_array
        }


time: 38.8 ms (started: 2023-04-13 21:48:03 +00:00)


Example

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 816 ms (started: 2023-04-13 21:48:06 +00:00)


In [30]:
path = "/content/drive/MyDrive/bispectrump_components/data/avgBL-Model.cif"

time: 493 µs (started: 2023-04-13 21:48:08 +00:00)


In [31]:
center_atom_id = 17
r_mu = 0.0779
R_cut = 0.25
input_file_path = path
output_directory = '/content/drive/MyDrive/bispectrump_components/data/'
#read data from file
input_data = get_INPUT_value(center_atom_id, r_mu, R_cut, input_file_path, output_directory, file_type='cif')
print(input_data)


{'r_ik': array([0.16062009, 0.12295675, 0.12219716, 0.12619947, 0.11925231]), 'theta_0': array([2.01841164, 1.54512013, 1.53557477, 1.58586934, 1.49856872]), 'theta': array([0.69919773, 0.3820468 , 1.18488014, 1.25583529, 1.15492866]), 'phi': array([0.049713  , 1.25332279, 5.41038219, 1.13378905, 6.18181825]), 'w_ik': array([1., 1., 1., 1., 1.]), 'delta': array([1., 1., 1., 1., 1.]), 'r_cut': array([0.25, 0.25, 0.25, 0.25, 0.25])}
time: 31.5 ms (started: 2023-04-13 21:48:09 +00:00)


---
# **Compute Bispectrum Component**


In [16]:
import numpy as np
import pandas as pd
import cmath


def fact(n):
    """
    This function is used to calculate factorial of a number by using
    an iterative approach instead of recursive approach
    """
    return np.prod(np.arange(1, n + 1))
class Wigner_D:
    """
    Args:
        j (scalar): angular momentum
        m (scalar): eigenvalue of angular momentum
        mp (scalar): eigenvalue of j along rotated axis
        theta_0 (scalar): first angle of rotation [0, pi]
        theta (scalar): second angle of rotation [0, pi]
        phi (scalar): third angle of rotation [0, 2*pi]
    Returns: complex number, Wigner D function
    ==========================Reference==================================
    [5] Chapter 4.3-(p.76,eq.1)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Quantum Theory of Angular Momentum (1988)
    """
    def __init__(self, j, m, mp, theta_0, theta, phi):
        if j < 0 or not np.isclose(j, int(j)) or (j % 1 == 0.5 and (m % 1 != 0 or mp % 1 != 0)):
            raise ValueError("Invalid input parameters: j must be a non-negative integer or half-integer, "
                             "m and mp must be between -j and j.")
        #if theta_0 < 0 or theta_0 > 2*np.pi or theta < 0 or theta > np.pi or phi < 0 or phi > 2 * np.pi:
            raise ValueError("Invalid input parameters: theta_0, theta, and phi must be within [0, pi] and [0, 2pi], respectively.")
            
        self.j = j
        self.m = m
        self.mp = mp
        self.theta_0 = theta_0
        self.theta = theta
        self.phi = phi
    def compute_dsmall(self):
        """
        This method is used to calculate the Wigner d small- real function involving trigonometric functions
        ==========================Reference==================================
        [5] Chapter 4.3.1-(p.76,eq.4)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Returns: Wigner d - real function
        """
        kmax = max(0, self.m - self.mp)
        kmin = min(self.j + self.m, self.j - self.mp)
        term1 = np.sqrt(fact(self.j + self.m) * fact(self.j - self.m) * fact(self.j + self.mp) * fact(self.j - self.mp))
        sum = 0
        for k in range(int(kmax), int(kmin) + 1):
            numerator = (-1) ** k * (cmath.cos(self.theta / 2)) ** (2 * self.j - 2 * k + self.m - self.mp) * \
                        (cmath.sin(self.theta / 2)) ** (2 * k - self.m + self.mp)
            denominator = fact(k) * fact(self.j + self.m - k) * fact(self.j - self.mp - k) * fact(self.mp - self.m + k)
            sum += numerator / denominator
        return sum*term1
    def wigner_D(self):
        #term1 = np.exp(-1j * self.m * self.theta_0)
        term1 = np.cos(self.m * self.theta_0) - 1j*(np.sin(self.m * self.theta_0))
        term2 = self.compute_dsmall()
        #term3 = np.exp(-1j * self.mp * self.phi)
        term3 = np.cos(self.mp * self.phi) -1j*(np.sin(self.mp * self.phi))
        result = term1 * term2 * term3
        return result
    def U_rot(self):
        '''
        Parameters:
        j: integer/half integer number angular momentum
        m: integer/half integer number
        Eigenvalue of angular momentum along rotated axis
        mp: eigenvalue of j along rotated axis
        mpp:
        theta_0: fist angle of rotation [0,pi]
        theta: second angle of rotation [0,pi]
        phi: third angle of rotation [0,2pi]
        rotational od a coordinate system through an angle theta_0
        about an axis n(theta,phi)
        Returns: Rotational matrix U
        ==========================Reference==================================
        [5] Chapter 4  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
              Quantum Theory of Angular Momentum (1988)
        '''
        mpp_vals = np.linspace(-j, j, int(2*j+1))
        U = 0
        for mpp in mpp_vals:
          WD_1 = Wigner_D(j, m, mpp, phi, theta, -phi)
          term1 = WD_1.wigner_D()
          term2 = np.cos(mpp * theta_0) - 1j*(np.sin(mpp * theta_0))
          WD_2 = Wigner_D(j, mpp, mp, phi, -theta, -phi)
          term3 = WD_2.wigner_D()
          Um_mp = term1 * term2 * term3
          U += Um_mp
        return U

class Clebsch_Gordan:
    """
    Definition:
    A Clebsch-Gordan coefficients are vector addition coefficients. They play an important role in decomposition of
    reducible representations of rotation. Let j1 and j2 with projections on m1 and m2 on the quantization axis.
    The coefficients represent the probability amplitude that j1 and j2 are coupled into a resultant angular momentum
    j with projection m.
    Args:
        j1 (scalar): angular momentum
        j2 (scalar): angular momentum
        j (scalar): angular momentum
        m1 (scalar): eigenvalue of angular momentum
        m2 (scalar): eigenvalue of angular momentum
        m (scalar): eigenvalue of angular momentum
    Returns: complex number, Clebsh Gordan function
    ==========================Reference==================================
    [5] Chapter 8 D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Quantum Theory of Angular Momentum (1988)
    [12] Chapter 3 Biedenharn, L., & Louck, J.D. ,
        Encyclopedia of Mathematics and its Applications (1981)
    """
    def __init__(self, j1, j2, j, m1, m2, m):
        self.j1 = j1
        self.j2 = j2
        self.j = j
        self.m1 = m1
        self.m2 = m2
        self.m = m
        self.J = j1 + j2 + j
        #Condition 1 & 2 & 5
        if not (abs(j1 - j2) <= j <= j1 + j2 and m1 + m2 == m and j1 + j2 - j % 1 != 0.5):
            raise ValueError("Invalid input parameters: j1, j2, j, m1, m2, and m must satisfy the triangle inequality.\ "
                             "j1+j2-j must not be a half-integer")
        #Condition 3 & 6
        if not all(abs(x) <= y and (x % 0.5 == 0 or x % 1 == 0) for x, y in zip([m1, m2, m], [j1, j2, j])):
            raise ValueError("Invalid input parameters: |m1| <= j1, |m_2| <= j2, |m| <= j\ "
                             "m1, m2, m must be integer or half-integer (positive or negative) numbers")
        # Condition 4
        J =(j1+j2+j)
        if J < (int(j1+j2+j)) and J <0:
            raise ValueError("Invalid input parameters: j1, j2, j must not exceed a positive integer J")
        # Condition 7
        if not all(
                isinstance(x, (int, float, np.integer, np.floating)) and x >= 0 and (x % 0.5 == 0 or x % 1 == 0) for x
                in [j1, j2, j]):
            raise ValueError(
                "Invalid input parameters: j1, j2, j must be integer or half-integer non-negative numbers")
        # Condition 8
        if not all(isinstance(x, (int, float, np.integer, np.floating)) and x >= 0 and x % 1 == 0 for x in
                   [j1 + m1, j2 + m2, j + m, j1 + j2 + j]):
            raise ValueError("Invalid input parameters: j1+m1,j2+m2,j+m,j1+j2+j must be integer non-negative numbers")
    def cg(self):
        if self.m1 + self.m2 != self.m:
            return 0.0  # delta function fails
        prefactor = cmath.sqrt((2 * self.j + 1) * fact(self.j + self.j1 - self.j2) * fact(self.j-self.j1 + self.j2) \
                               * fact(self.j1 + self.j2 - self.j) / fact(self.j + self.j1 + self.j2 + 1))
        coefficient = cmath.sqrt(fact(self.j + self.m) * fact(self.j - self.m) / (fact(self.j1 + self.m1) \
                               * fact(self.j1 - self.m1) * fact(self.j2 + self.m2) * fact(self.j2 - self.m2)))
        sum = 0.0
        smin= max(0, int(self.m1-self.j1),int(self.j2-self.j1+self.m))
        smax= min(int(self.j2+self.j+self.m1),int(self.j-self.j1+self.j2),\
                   int(self.j+self.m))

        for s in range(smin,smax+1):
            den = fact(s) * fact(self.j - self.j1 + self.j2 - s) * fact(self.j + self.m - s) \
                  * fact(self.j1 - self.j2 - self.m + s)
            num = ((-1) ** (self.j2 + self.m2 + s))* fact(self.j2 + self.j + self.m1 - s) * fact(self.j1 - self.m1 + s)
            sum += num / den
        cg = prefactor * coefficient * sum
        return cg
def H_coeff(j1,j2,j,m1,m2,m,m1p,m2p,mp):
    '''
    This function calculate coupling coefficient H via computing
    the Clebsch-Gordan coefficient for cg(j1,m1,j2,m2,j,m)
    and cg(j1,m1p,j2,m2p,j,mp)
    Parameters:
        j1: angular momentum 1
        j2: angular momentum 2
        j: total angular momentum (j1+j2)
        m1: eigenvalue of angular momentum j1
        m2: eigenvalue of angular momentum j2
        m: eigenvalue of angular momentum j
        m1p: eigenvalue of j1 along rotated axis
        m2p: eigenvalue of j2 along rotated axis
        mp: eigenvalue of j along rotated axis
    Returns: Coupling coefficient H(j1,j2,j,m1,m2,m.m1p,m2p,mp)
    ======================Reference=========================
    [1] Thompson, Swiler, Trott, Foiles, Tucker,
        Spectral neighbor analysis method for automated generation of quantum-accurate interatomic potentials (2015)
    [5] Chapter 8  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Quantum Theory of Angular Momentum (1988)
    '''
    CG = Clebsch_Gordan(j1,j2,j,m1,m2,m)
    cg = CG.cg()
    CGp = Clebsch_Gordan(j1,j2,j,m1p,m2p,mp)
    cg_p = CGp.cg()
    H = (cg)*(cg_p)
    return H

class Bispectrum:
    """
    Calculate bispectrum components
    """
    def __init__(self, j, j1, j2, input_data):
        '''
            j: j index
            j1: j1 index
            j2: j2 index
            input_data: input data dictionary with extracted values:
            r_ik (array): dictance from center atom to n neighbor atom, dim = [n,]
            theta_0 (array): first angle of rotation [0, pi] , dim = [n,]
            theta (array): second angle of rotation [0, pi], dim = [n,]
            phi (array): third angle of rotation [0, 2pi], dim = [n,]
            w_ik (array): weight coefficient, dim = [n,]
            delta (array): delta function, dim = [n,]
            r_cut (array): cutoff distance, dim = [n,]
        '''
        self.j = j
        self.j1 = j1
        self.j2 = j2
        self.input_val = input_data #dictionary with values

        #Condition check
        if not (abs(j1 - j2) <= j <= j1 + j2 and j1 + j2 - j % 1 != 0.5):
            raise ValueError("Invalid input parameters: j1, j2, j must satisfy the triangle inequality.\ "
                             "j1+j2-j must not be a half-integer")
        J = (j1 + j2 + j)
        if J < (int(j1 + j2 + j)) and J < 0:
            raise ValueError("Invalid input parameters: j1, j2, j must not exceed a positive integer J")

        

time: 3.73 ms (started: 2023-04-13 21:44:02 +00:00)


In [17]:
j=2
m=1
mp=1
theta_0=np.pi/6
theta=4*np.pi/3
phi=np.pi/2
wD = Wigner_D(j,m,mp,theta_0,theta,phi)
U = wD.U_rot()

print(U)

(0.7321313509461094+0.1997595264191645j)
time: 8.24 ms (started: 2023-04-13 21:44:07 +00:00)


Example